In [1]:
input=sc.textFile("input.txt").map(lambda line:line.split("\t")).filter(lambda x:x!="").map(lambda wordlist:(wordlist[0],wordlist[1]))
maxIteration=10
def computeContrib(neighbors,rank):
    numNeighbors=len(neighbors)
    for neighbor in neighbors:
        yield (neighbor,rank/numNeighbors)

grouped=input.groupByKey()
countsNeighbors=grouped.mapValues(lambda x:len(x))
rank=input.map(lambda pair:(pair[0],1.0)).distinct()

In [2]:
for iteration in range(int(maxIteration)):
    groupedRank=grouped.join(rank)
    #joined set will be(nodeid,(neighborlist,rank))
    contrib=groupedRank.flatMap(lambda node:computeContrib(node[1][0],node[1][1]))
    #computeContrib set will be (nodeid,contibutionRank)
    rank=contrib.reduceByKey(lambda a,b:a+b).mapValues(lambda rank: rank * 0.85 + 0.15)
rankDict=rank.collectAsMap()

In [7]:
Top20Nodes=rank.sortBy(lambda x:x[1],ascending=False)
print(Top20Nodes.take(20))

[('41909', 445.717785968565), ('597621', 406.6283667503004), ('504140', 399.08930874749024), ('384666', 392.8258437305223), ('537039', 383.90912550319206), ('486980', 382.16565078374185), ('751384', 361.8135365307977), ('32163', 361.4220959849127), ('163075', 357.91973093907956), ('605856', 356.5971554341694), ('765334', 349.9418245162527), ('558791', 343.49196417404994), ('425770', 333.0037797186571), ('173976', 328.6989370004975), ('908351', 314.9342171185025), ('905628', 296.6611098281252), ('459074', 295.29521476429824), ('551829', 290.5874145390438), ('452291', 288.18733307824635), ('687325', 285.7872834876148)]


In [10]:
Top20Nodes=sc.parallelize(Top20Nodes.take(20))
TopCountNeighbors=Top20Nodes.join(countsNeighbors).map(lambda pair:(pair[0],pair[1][1]))
print(TopCountNeighbors.collect())

[('558791', 22), ('425770', 28), ('905628', 7), ('537039', 27), ('486980', 6), ('551829', 13), ('605856', 22), ('597621', 22), ('504140', 19), ('908351', 24), ('459074', 23), ('452291', 3), ('751384', 68), ('41909', 10), ('384666', 20), ('32163', 32), ('765334', 4), ('163075', 36), ('173976', 26), ('687325', 14)]


In [11]:
TopNeighbors=Top20Nodes.join(grouped).flatMapValues(lambda x:list(x[1])).mapValues(lambda a:rankDict[a]).reduceByKey(lambda a,b:a+b)
TopNeighbors=TopNeighbors.join(TopCountNeighbors).mapValues(lambda x:x[0]/x[1])
print(TopNeighbors.collect())

[('425770', 22.607100420901673), ('905628', 101.80211497812395), ('551829', 19.798252497215252), ('597621', 26.83392592962629), ('908351', 29.863083781005994), ('459074', 35.19453797297566), ('751384', 11.01570181045522), ('384666', 53.464889811430794), ('765334', 195.4290546343321), ('173976', 19.299330451381902), ('558791', 21.942239357812852), ('537039', 24.62369923391188), ('486980', 62.323118479301506), ('605856', 25.356522198556277), ('504140', 37.694216024789824), ('452291', 213.63285696544196), ('41909', 61.71997839787936), ('32163', 33.129743259829134), ('163075', 22.567096657679247), ('687325', 53.84053165111204)]
